In [1]:
import pandas as pd
import polars as pl
from IPython.display import display
from bokeh.plotting import show
from bokeh.io import output_notebook
from cfa_analysis.data_retrieval import (
    get_data_from_imf,
    rename_from_abbr_to_full_name,
    get_all_metric_data,
    get_cfa_and_noncfa_data,
    get_country_mapping,
    get_indicators_data,
    get_imf_data_df,
    get_all_metric_data
)
from cfa_analysis.constants import CFA_FRANC_ZONE, WEST_AFRICA, MIDDLE_AFRICA
from cfa_analysis.data_cleanup import (
    rename_from_abbr_to_full_name, 
    find_outliers_IQR, 
    clean_up_indicators_dict,
    find_duplicate_indicators
)
from cfa_analysis.data_classes import Indicator
from cfa_analysis.presentation import generate_graph, chat_gpt_analyze_results
from cfa_analysis.analysis import analyze_medians, get_median_df
import panel as pn
from typing import Union, Tuple, Dict, List
%load_ext jupyter_ai_magics
output_notebook()
pn.extension()

Loading BokehJS ...

In [65]:
indicators = get_indicators_data()

In [66]:
indicators = clean_up_indicators_dict(indicators)

In [67]:
indicators

{'NGDP_RPCH': {'label': 'Real GDP Growth Rate',
  'description': "Gross domestic product is the most commonly used single measure of a country's overall economic activity. It represents the total value at constant prices of final goods and services produced within a country during a specified time period, such as one year.",
  'source': 'World Economic Outlook (October 2023)',
  'unit': 'Annual % change',
  'dataset': 'WEO'},
 'NGDPD': {'label': 'GDP, Current Prices',
  'description': "Gross domestic product is the most commonly used single measure of a country's overall economic activity. It represents the total value at current prices of final goods and services produced within a country during a specified time period, such as one year.",
  'source': 'World Economic Outlook (October 2023)',
  'unit': 'Billions of U.S. dollars',
  'dataset': 'WEO'},
 'NGDPDPC': {'label': 'GDP Per Capita, Current Prices',
  'description': "Gross domestic product is the most commonly used single measure

In [ ]:
label_value = indicator_info.get("label")

label = label_value.strip("\n") if label_value is not None else None

indicator = Indicator(
    indicator_abbrv,
    indicator_info.get("description", ""),
    label,
    indicator_info.get("unit", "").strip("\n"),
    indicator_info.get("dataset", "")
)

In [ ]:
test = {'': {'label': None,
  'description': None,
  'source': None,
  'unit': None,
  'dataset': None}
 }

In [ ]:
indicator_info = test.get('', {})

In [ ]:
indicator_info

In [ ]:
indicator_info.get('label', "")

In [ ]:
indicators

In [29]:
duplicate_combinations = find_duplicate_indicators(indicators)

In [30]:
duplicate_combinations

{('Real GDP Growth Rate', 'Annual % change'): ['NGDP_RPCH', 'NGDP_R_PCH'],
 ('Government Revenue', '% of GDP'): ['rev', 'GGR_G01_GDP_PT'],
 ('Government Expenditure', '% of GDP'): ['exp', 'GGX_GDP']}

In [42]:
all_countries, countries,  = get_country_mapping()
indicators = clean_up_indicators_dict(get_indicators_data())

In [6]:
indicators

{'NGDP_RPCH': {'label': 'Real GDP Growth Rate',
  'description': "Gross domestic product is the most commonly used single measure of a country's overall economic activity. It represents the total value at constant prices of final goods and services produced within a country during a specified time period, such as one year.",
  'source': 'World Economic Outlook (October 2023)',
  'unit': 'Annual % change',
  'dataset': 'WEO'},
 'NGDPD': {'label': 'GDP, Current Prices',
  'description': "Gross domestic product is the most commonly used single measure of a country's overall economic activity. It represents the total value at current prices of final goods and services produced within a country during a specified time period, such as one year.",
  'source': 'World Economic Outlook (October 2023)',
  'unit': 'Billions of U.S. dollars',
  'dataset': 'WEO'},
 'NGDPDPC': {'label': 'GDP Per Capita, Current Prices',
  'description': "Gross domestic product is the most commonly used single measure

In [60]:
indicator_abbrv = "GGXCNL_NGDP"

In [61]:
indicator_info = indicators.get(indicator_abbrv, {})
indicator_label = indicator_info.get("label", "")
indicator_label = indicator_label.strip("\n") if indicator_label is not None else None
indicator_unit = indicator_info.get("unit", "")
indicator_unit = indicator_unit.strip("\n") if indicator_unit is not None else None
indicator = Indicator(indicator_abbrv, indicator_info.get("description", ""), indicator_label, indicator_unit, indicator_info.get("source", ""))

In [48]:
indicator

Indicator(abbrv='SITC1_1', description='description if any. Otherwise, this can be the same as the label', label='Beverages, Tobacco', unit='Index', source='IMF Board Policy Paper "Sustaining Long-Run Growth and Macroeconomic Stability in Low-Income Countries—The Role of Structural Transformation and Diversification", March 2014.')

In [62]:
all_data_df = get_imf_data_df(
                    get_cfa_and_noncfa_data(
                        indicator.abbrv, 
                        countries, 
                        all_countries
                    ), 
                    indicator.label
                )

In [50]:
all_data_df

Country,Year,"Beverages, Tobacco"
str,i32,f32
"""Benin""",1980,0.693716
"""Burkina Faso""",1980,null
"""Cameroon""",1980,1.022071
"""Central Africa…",1980,0.5346
"""Chad""",1980,0.646948
"""Congo, Republi…",1980,0.721845
"""Côte d'Ivoire""",1980,0.50612
"""Equatorial Gui…",1980,0.252849
"""Gabon""",1980,null


In [64]:
len(all_data_df.select(pl.col('Country', indicator.label)).filter((pl.col("Country").is_in(CFA_FRANC_ZONE)) & (pl.col(indicator.label).is_null())))

162

In [24]:
all_data_df.null_count().select(pl.col("Country").where(pl.col("Country").is_in(CFA_FRANC_ZONE)))

InvalidOperationError: `is_in` cannot check for Utf8 values in UInt32 data

In [20]:
all_data_df.null_count().select(pl.col('Country', "Real GDP Growth Rate").where(pl.col("Country").is_in(CFA_FRANC_ZONE)))['Real GDP Growth Rate'][0]

InvalidOperationError: `is_in` cannot check for Utf8 values in UInt32 data

In [18]:
get_all_metric_data(CFA_FRANC_ZONE, 'SITC1_4', countries)

{'BEN': {'1963': 0.47202349,
  '1964': 0.50725824,
  '1965': 0.44623899,
  '1966': 0.69657379,
  '1967': 0.68824965,
  '1968': 0.68442166,
  '1969': 0.73645264,
  '1970': 0.71534765,
  '1971': 0.73477888,
  '1972': 0.81639701,
  '1973': 0.84191048,
  '1974': 0.83246547,
  '1975': 0.66899866,
  '1976': 0.77950549,
  '1977': 0.74640614,
  '1978': 0.7689923,
  '1979': 0.76493913,
  '1980': 0.65878862,
  '1981': 0.73172027,
  '1982': 0.69963235,
  '1983': 0.78297615,
  '1984': 0.65716678,
  '1985': 0.44611275,
  '1986': 0.46400967,
  '1987': 0.81052828,
  '1988': 0.52151352,
  '1989': 0.35887975,
  '1990': 0.57659483,
  '1991': 0.24556597,
  '1992': 0.48120239,
  '1993': 0.29713026,
  '1994': 0.51201302,
  '1996': 0.6110661,
  '1998': 0.73334897,
  '1999': 0.84817511,
  '2000': 0.83434296,
  '2001': 0.83764988,
  '2002': 0.86420518,
  '2003': 0.82316369,
  '2004': 0.8099522,
  '2005': 0.86051506,
  '2006': 0.60369092,
  '2007': 0.72856307,
  '2008': 0.76253724,
  '2009': 0.58868539,
  '201

In [83]:
get_all_metric_data(WEST_AFRICA, indicator_abbrv, countries)

{'AGO': {'2003': 0.27238269105202,
  '2004': 0.51890979243206,
  '2005': 0.8443750889435,
  '2006': 2.0081841759453,
  '2007': 1.7026305887153,
  '2008': 1.6238506616282,
  '2009': 1.2316119159191,
  '2010': 2.0824652647589,
  '2011': 2.4751411094605,
  '2012': 2.8061259508176,
  '2013': 2.619415950936,
  '2014': 2.072143172031,
  '2015': 2.5725874988662,
  '2016': 3.796627623062,
  '2017': 2.5803248239314,
  '2018': 2.4985564821465,
  '2019': 3.087356407648,
  '2020': 3.9294429354907,
  '2021': 3.2916805319307,
  '2022': 2.0375894672237,
  '2023': 2.2932781928364}}

In [73]:
pl.Config.set_tbl_rows(-1)

polars.config.Config

In [74]:
all_data_df

Country,Year,Portfolio Investment Assets
str,i32,f32
"""Benin""",1995,64.199997
"""Burkina Faso""",1995,null
"""Cameroon""",1995,26.16
"""Central Africa…",1995,null
"""Chad""",1995,null
"""Congo, Republi…",1995,2.01
"""Côte d'Ivoire""",1995,8.41
"""Equatorial Gui…",1995,0.0
"""Gabon""",1995,29.83


In [30]:
median_df = get_median_df(all_data_df, indicator.label)

In [31]:
median_df

Year,cfa_median,noncfa_median
i32,f32,f32
1980,-204.100006,-4.9
1981,-174.899994,-3.9
1982,-184.899994,-2.0
1983,-154.600006,-1.2
1984,-114.400002,-1.0
1985,-45.25,-1.2
1986,-51.200001,-1.4
1987,-81.650002,-1.0
1988,-4.3,-1.2


In [34]:
median_df = median_df.with_columns(pl.col("cfa_median").abs().alias('abs_cfa_median'),
                pl.col("noncfa_median").abs().alias('abs_noncfa_median')
                )

In [35]:
median_df

Year,cfa_median,noncfa_median,abs_cfa_median,abs_noncfa_median
i32,f32,f32,f32,f32
1980,-204.100006,-4.9,204.100006,4.9
1981,-174.899994,-3.9,174.899994,3.9
1982,-184.899994,-2.0,184.899994,2.0
1983,-154.600006,-1.2,154.600006,1.2
1984,-114.400002,-1.0,114.400002,1.0
1985,-45.25,-1.2,45.25,1.2
1986,-51.200001,-1.4,51.200001,1.4
1987,-81.650002,-1.0,81.650002,1.0
1988,-4.3,-1.2,4.3,1.2


In [ ]:
p = generate_graph(median_df.to_pandas(), indicator_label, indicator_unit)

In [12]:
from bokeh.plotting import figure, show
from bokeh.models import (
    Legend,
    HoverTool,
    PrintfTickFormatter,
    ColumnDataSource,
    Range1d,
    Title,
    BasicTickFormatter,
    Range1d
)

In [60]:
p = figure(
    x_axis_label="Year",
    y_axis_label=f"{indicator.unit}",
    width=800,
    height=400,
    toolbar_location=None,
    y_axis_type="log"
)

In [61]:
p.line(
        x="Year",
        y="noncfa_median",
        color="#D55E00",
        line_width=3,
        legend_label="Non-CFA",
        source=ColumnDataSource(median_df.to_dict(as_series=False)),
        line_alpha=0.8,
    )
p.line(
        x="Year",
        y="cfa_median",
        color="#0072B2",
        line_width=3,
        legend_label="CFA",
        source=ColumnDataSource(median_df.to_dict(as_series=False)),
        line_alpha=0.8,
    )
p.line(
        x="Year",
        y="abs_noncfa_median",
        color="#D55E00",
        line_width=2,
        source=ColumnDataSource(median_df.to_dict(as_series=False)),
        line_alpha=0.3,
    )
p.line(
        x="Year",
        y="abs_cfa_median",
        color="#0072B2",
        line_width=2,
        source=ColumnDataSource(median_df.to_dict(as_series=False)),
        line_alpha=0.3,
    )

GlyphRenderer(id='9a94e4f5-926e-4036-88ec-9e684b35a48b', ...)

In [62]:
for legend in p.legend:
    p.add_layout(legend, "right")

hover = HoverTool(
    tooltips=[
        ("Year", "@Year"),
        (f"Median {indicator.label} (Non-CFA)", "@noncfa_median{0.00}"),
        (f"Median {indicator.label} (CFA)", "@cfa_median{0.00}"),
    ]
)
p.add_tools(hover)

p.add_layout(
    Title(
        text="CFA African Countries vs. Non-CFA African Countries\n\n",
        text_font_size="12pt",
        text_align="center",
        align="center",
        text_font_style="normal",
    ),
    "above",
)
p.add_layout(
    Title(
        text=f"Median {indicator.label}",
        text_font_size="18pt",
        text_align="center",
        align="center",
    ),
    "above",
)
p.title.offset = 200
p.title.align = "center"

# p.xgrid.grid_line_color = "#DDDDDD"
# p.ygrid.grid_line_color = "#DDDDDD"

p.axis.minor_tick_line_color = None  # turn off x-axis minor ticks
# p.axis.major_tick_line_alpha =  0.1
p.axis.major_tick_line_color = "#AAAAAA"
p.axis.major_tick_line_dash = "dashed"

p.axis.axis_label_text_font_size = "12pt"
p.axis.axis_label_text_font_style = "bold"
p.axis.major_label_text_font_size = "12px"
p.axis.axis_label_standoff = 20
p.xaxis.major_label_orientation = 1.0

p.legend.border_line_color = None
p.legend.border_line_alpha = 0
p.legend.click_policy = "hide"

p.min_border = 100

In [63]:
show(p)

In [19]:
test = median_df.to_pandas()

In [21]:
test['noncfa_median'] = abs(test['noncfa_median'])

In [22]:
test

,Year,cfa_median,noncfa_median
0,1980,-204.100006,4.90
1,1981,-174.899994,3.90
2,1982,-184.899994,2.00
3,1983,-154.600006,1.20
4,1984,-114.400002,1.00
5,1985,-45.250000,1.20
6,1986,-51.200001,1.40
7,1987,-81.650002,1.00
8,1988,-4.300000,1.20
9,1989,-0.700000,0.90


In [11]:
import polars.selectors as cs

In [13]:
all_data_df.melt(
        id_vars="Country",
        value_vars=cs.numeric(),
        variable_name="Year",
        value_name=indicator.label,
    )df.cast({"Year": pl.UInt8, "bar": pl.UInt8})

Country,Year,General Government Net Lending/borrowing
str,str,f64
"""Benin""","""1980""",null
"""Burkina Faso""","""1980""",null
"""Central Africa…","""1980""",null
"""Côte d'Ivoire""","""1980""",null
"""Cameroon""","""1980""",null
"""Congo, Republi…","""1980""",null
"""Gabon""","""1980""",null
"""Guinea-Bissau""","""1980""",null
"""Equatorial Gui…","""1980""",-204.1


In [15]:
all_data_df.columns

['Country',
 '1980',
 '1981',
 '1982',
 '1983',
 '1984',
 '1985',
 '1986',
 '1987',
 '1988',
 '1989',
 '1990',
 '1991',
 '1992',
 '1993',
 '1994',
 '1995',
 '1996',
 '1997',
 '1998',
 '1999',
 '2000',
 '2001',
 '2002',
 '2003',
 '2004',
 '2005',
 '2006',
 '2007',
 '2008',
 '2009',
 '2010',
 '2011',
 '2012',
 '2013',
 '2014',
 '2015',
 '2016',
 '2017',
 '2018',
 '2019',
 '2020',
 '2021',
 '2022',
 '2023']

In [36]:
median_df = (
        all_data_df.group_by(["Year"], maintain_order=True)
        .agg(
            pl.col(indicator_label)
            .where(pl.col("Country").is_in(CFA_FRANC_ZONE))
            .median()
        )
        .join(
            all_data_df.group_by(["Year"], maintain_order=True).agg(
                pl.col(indicator_label)
                .where(
                    pl.col("Country").is_in(WEST_AFRICA)
                    | pl.col("Country").is_in(MIDDLE_AFRICA)
                )
                .median()
            ),
            on="Year",
        )
    ).drop_nulls().rename({indicator_label: 'cfa_median', f"{indicator_label}_right": 'noncfa_median'})

In [40]:
median_df = get_median_df(all_data_df, indicator.label)

In [41]:
median_df

Year,cfa_median,noncfa_median
str,f64,f64
"""1980""",-204.1,-4.9
"""1981""",-174.9,-3.9
"""1982""",-184.9,-2.0
"""1983""",-154.6,-1.2
"""1984""",-114.4,-1.0
"""1985""",-45.25,-1.2
"""1986""",-51.2,-1.4
"""1987""",-81.65,-1.0
"""1988""",-4.3,-1.2


In [10]:
pl.Config.set_tbl_rows(-1)

polars.config.Config

In [45]:
test = median_df.to_pandas()

In [48]:
test['Year'] = test['Year'].astype(int)

In [49]:
test

,Year,cfa_median,noncfa_median
0,1980,-204.10,-4.90
1,1981,-174.90,-3.90
2,1982,-184.90,-2.00
3,1983,-154.60,-1.20
4,1984,-114.40,-1.00
5,1985,-45.25,-1.20
6,1986,-51.20,-1.40
7,1987,-81.65,-1.00
8,1988,-4.30,-1.20
9,1989,-0.70,-0.90


In [ ]:
data = median_df.to_dict(as_series=False)

In [ ]:
data['cfa_median'] = [round(value, 1) for value in data['cfa_median']]
data['noncfa_median'] = [round(value, 1) for value in data['noncfa_median']]

In [ ]:
data

In [50]:
p = generate_graph(test, indicator.label, indicator.unit)

In [51]:
show(p)

In [ ]:
from bokeh.models import (ColumnDataSource, Range1d)
from bokeh.plotting import figure, show

In [ ]:
p = figure(
        x_axis_label="Year",
        y_axis_label=f"{indicator.unit}",
        width=800,
        height=400,
        toolbar_location=None
    )

In [ ]:
p.line(
    x="Year",
    y="noncfa_median",
    color="#D55E00",
    source=ColumnDataSource(data=median_df.to_pandas()),
    line_alpha=0.7,
)

In [ ]:
show(p)

In [ ]:
process_single_indicator(all_data_df, indicator.label, indicator.unit, indicator.description)

In [ ]:
if (indicator_label,unit) in duplicate_combinations:
    merged_df = merge_duplicate_dfs(get_all_duplicate_dfs(duplicate_combinations, indicator_label, unit,indicators), indicator_label)

In [ ]:
def get_all_duplicate_dfs(duplicate_combinations: Dict[Tuple[str, str], List[str]], indicator_label: str, unit: str, indicators: Dict[str,Dict[str,str]]):
    all_dfs = []
    for indicator_abbrv in duplicate_combinations[(indicator_label,unit)]:
            all_dfs.append(get_imf_data_df(get_cfa_and_noncfa_data(
                indicator_abbrv, countries, all_countries
            ), indicator_label))
            indicators.pop(indicator_abbrv)
    return all_dfs
    
def merge_duplicate_dfs(all_dfs: List[pl.DataFrame], indicator_label: str) -> pl.DataFrame:
    merged_df = all_dfs[0]
    for x,df in enumerate(all_dfs):
        if x == len(all_dfs) - 1: 
            continue
        merged_df = merged_df.join(df, on=['Country', 'Year'], how='outer').rename({indicator_label: f"{indicator_label}_left"})
        merged_df = (
            merged_df
            .with_columns([
                pl.when(pl.col(f"{indicator_label}_left").is_not_null() & pl.col(f"{indicator_label}_right").is_not_null())
                .then((pl.col(f"{indicator_label}_left").add(pl.col(f"{indicator_label}_right"))).truediv(2))
                .when(pl.col(f"{indicator_label}_left").is_not_null())
                .then(pl.col(f"{indicator_label}_left"))
                .when(pl.col(f"{indicator_label}_right").is_not_null())
                .then(pl.col(f"{indicator_label}_right"))
                .otherwise(None)
                .alias(indicator_label)
            ])
        ).drop([f"{indicator_label}_left", f"{indicator_label}_right"])
    return merged_df

In [ ]:
merged_df

In [ ]:
# cleaning up 'NGDP_RPCH'

In [ ]:
data_1 = rename_from_abbr_to_full_name(
        get_all_metric_data(
            MIDDLE_AFRICA, 
            'NGDP_RPCH', 
            countries
        ),
    all_countries
    )

In [ ]:
data_2 = rename_from_abbr_to_full_name(
        get_all_metric_data(
            MIDDLE_AFRICA, 
            'NGDP_R_PCH', 
            countries
        ),
    all_countries
    )

In [ ]:
label = indicators['NGDP_RPCH']['label']

In [ ]:
df_1 = get_imf_data_df(data_1, label)

In [ ]:
df_1

In [ ]:
df_2 = get_imf_data_df(data_2, label)

In [ ]:
merged_df = df_1.join(df_2, on=['Country', 'Year'], how='outer').rename({"Real Gdp Growth Rate": "Real Gdp Growth Rate_left"})

In [ ]:
merged_df = (
    merged_df
    .with_columns([
        pl.when(pl.col('Real Gdp Growth Rate_left').is_not_null() & pl.col('Real Gdp Growth Rate_right').is_not_null())
        .then((pl.col('Real Gdp Growth Rate_left').add(pl.col('Real Gdp Growth Rate_right'))).truediv(2))
        .when(pl.col('Real Gdp Growth Rate_left').is_not_null())
        .then(pl.col('Real Gdp Growth Rate_left'))
        .when(pl.col('Real Gdp Growth Rate_right').is_not_null())
        .then(pl.col('Real Gdp Growth Rate_right'))
        .otherwise(None)
        .alias('Real Gdp Growth Rate')
    ])
).drop(["Real Gdp Growth Rate_left", "Real Gdp Growth Rate_right"])

In [ ]:
merged_df

In [ ]:
merged_df